# Virtual Machine Workflow Agent

## Overview

This notebook demonstrates how to build an AI-powered workflow agent that interacts with vSphere, lists available virtual machines (VMs), retrieves their details, and processes commands using a machine learning model. The agent also integrates with custom tools and persists workflow data using SQLite. Each step is carefully broken down, and instructions are provided for users who may be new to the components involved.

## 1. Setup

### Installing Required Packages

Before we begin, we need to install several Python libraries that are critical for this workflow. These libraries will allow us to:
- **`langchain`**: A framework for building language models and agents.
- **`langgraph`**: A library for creating AI-driven workflows.
- **`requests`**: For sending HTTP requests.
- **`pyvmomi`**: For interacting with the vSphere API.
- **`dotenv`**: For managing environment variables.

The `%pip install` command ensures all necessary packages are installed.

In [22]:
%pip install langchain langgraph langgraph-checkpoint-sqlite requests termcolor pyvmomi python-dotenv

Note: you may need to restart the kernel to use updated packages.


## 2. Building Blocks

### vSphere Configuration

Here, we set up the configuration for connecting to vSphere, which is VMware's cloud infrastructure platform. We use environment variables to securely store connection details, such as the vSphere host, username, and password. These are loaded using the `dotenv` library to avoid hardcoding sensitive information in the script.

In [23]:
from dotenv import load_dotenv
import os

load_dotenv()

vsphere_config = {
    "host": os.getenv("VSPHERE_HOST"),
    "username": os.getenv("VSPHERE_USERNAME"),
    "password": os.getenv("VSPHERE_PASSWORD"),
}

### Testing Connectivity to vSphere

In this section, we test the connection to the vSphere environment using the credentials provided in the previous configuration step. The goal is to ensure that we can successfully connect to vSphere, retrieve all virtual machines (VMs), and print their names. We use the following methods from the `vsphere_utils` module:

- **`connect_to_vsphere`**: Establishes a connection to the vSphere environment.
- **`get_all_vms`**: Retrieves a list of all VMs in the connected vSphere environment.
- **`disconnect_from_vsphere`**: Safely disconnects from vSphere to free up resources.

**Steps**:
1. **Connect to vSphere** using the host, username, and password stored in the `vsphere_config`.
2. **Retrieve all VMs** using the `get_all_vms` function, which returns a list of VMs and their names.
3. **Handle errors**: If there's an issue connecting to vSphere or retrieving VMs, an exception will be caught, and an error message will be displayed.
4. **Disconnect**: After the operation is complete (or if an error occurs), the connection to vSphere is closed using `disconnect_from_vsphere`.


In [24]:
from utils.vsphere.vsphere_utils import (
    get_all_vms,
    connect_to_vsphere,
    disconnect_from_vsphere,
)

si = None
try:
    # Connect to vSphere
    si, content = connect_to_vsphere(
        host=vsphere_config["host"],
        user=vsphere_config["username"],
        pwd=vsphere_config["password"],
    )

    vms, vm_names = get_all_vms(si, content)

except Exception as e:
    print(f"Failed to list vms. {str(e)}")

finally:
    if si:
        disconnect_from_vsphere(si)

Connected successfully to vCenter at vcsrs00-vc.infra.demo.redhat.com
VMs in the vSphere environment: roadshow-tpl-winweb01, winweb01, database, winweb02, haproxy, rhel86-tpl, rhel9-tpl, roadshow-tpl-database, rhel93-tpl, roadshow-tpl-winweb02


## 3. Configuring a Simple Model

In this section, we configure the machine learning model that our agent will use to process tasks. The `ModelService` class manages the interaction with the model (in this case, "llama3.1:8b-instruct-fp16"), allowing the agent to handle tasks such as listing VMs and retrieving details.

### Model Configuration

We initialize the `ModelService` with a specific model configuration, including parameters such as model endpoint, temperature (for controlling randomness), and others. This step enables our agent to perform model-based tasks using the provided configuration.


In [25]:
from services.model_service import ModelService

# Initialize the service with the model configuration
ollama_service = ModelService(model="llama3.1:8b-instruct-fp16")

## 4. Custom Tools

This section explains how we integrate custom tools into the agent’s workflow. These tools allow the agent to perform specific tasks like listing VMs or retrieving their details from the vSphere infrastructure. We use the `@tool` decorator from `langchain` to define tools that will interact with vSphere.

### Tool: `list_vms`

This tool retrieves a list of all available virtual machines (VMs) from vSphere. It connects to the vSphere API, fetches VM data, and returns a list of VM names. If the process fails, it returns an error message.

### Tool: `retrieve_vm_details`

This tool retrieves detailed information about a specific VM by its name, such as its operating system, resource allocation, and network configuration. The tool connects to the vSphere API, finds the specified VM, and fetches its details. If the VM is not found or there’s an issue, it returns an error message.


In [26]:
from langchain.tools import tool
from typing import Union, List, Dict
from utils.vsphere.vsphere_utils import (
    get_all_vms,
    connect_to_vsphere,
    disconnect_from_vsphere,
    get_vm_details,
    get_vm_by_name,
)


@tool(parse_docstring=True)
def list_vms() -> Union[List[str], str]:
    """
    A wrapper around a vSphere utility for listing all available virtual machines (VMs). Useful for retrieving a list of VMs from the connected vSphere environment.

    Returns:
        vms: A list of VM names or an error message if the operation fails.
    """
    si = None
    try:
        # Connect to vSphere
        si, content = connect_to_vsphere(
            host=vsphere_config["host"],
            user=vsphere_config["username"],
            pwd=vsphere_config["password"],
        )

        vms, vm_names = get_all_vms(si, content)
        return vm_names

    except Exception as e:
        return f"Failed to list vms. {str(e)}"

    finally:
        if si:
            disconnect_from_vsphere(si)


@tool(parse_docstring=True)
def retrieve_vm_details(vm_name: str) -> Union[Dict[str, Union[str, int, list]], str]:
    """
    A wrapper around a vSphere utility for extracting detailed information about a specific virtual machine (VM). Useful for retrieving the VM's operating system, resource allocations, and network configurations based on the provided VM name.

    Args:
        vm_name: The name of the virtual machine to retrieve details for.

    Returns:
        vm_details: A dictionary containing VM details or an error message if the operation fails or the VM is not found.
    """
    si = None
    try:
        # Connect to vSphere
        si, content = connect_to_vsphere(
            host=vsphere_config["host"],
            user=vsphere_config["username"],
            pwd=vsphere_config["password"],
        )

        # Find the VM by name
        vm = get_vm_by_name(content, vm_name)

        if not vm:
            return f"VM '{vm_name}' not found."

        # Get detailed information about the found VM
        vm_details = get_vm_details(vm)
        return vm_details

    except Exception as e:
        return f"Failed to retrieve details for VM '{vm_name}': {str(e)}"

    finally:
        if si:
            disconnect_from_vsphere(si)


In [27]:
from utils.general.tools import get_tools_name, get_tools_description

tools = [list_vms, retrieve_vm_details]

tools_name = get_tools_name(tools)
tools_description = get_tools_description(tools)

tools_description

"list_vms() -> Union[List[str], str] - A wrapper around a vSphere utility for listing all available virtual machines (VMs). Useful for retrieving a list of VMs from the connected vSphere environment., args: {{}}\nretrieve_vm_details(vm_name: str) -> Union[Dict[str, Union[str, int, list]], str] - A wrapper around a vSphere utility for extracting detailed information about a specific virtual machine (VM). Useful for retrieving the VM's operating system, resource allocations, and network configurations based on the provided VM name., args: {{'vm_name': {{'title': 'Vm Name', 'description': 'The name of the virtual machine to retrieve details for.', 'type': 'string'}}}}"

## 5. Workflow Graph Setup

### Node: `react_node_function`

This function handles the execution of tasks by the ReAct agent. It takes the current state of the agent and processes the user request using the available tools (like listing VMs or retrieving details). The response from the ReAct agent is returned as the output.


In [28]:
from state.agent_graph import AgentGraphState
from agent.react_agent import ReactAgent

def react_node_function(state: AgentGraphState):
    react_agent = ReactAgent(
        state=state,
        role="REACT_AGENT",
        tools=tools,
        ollama_service=ollama_service,
    )

    return react_agent.react(user_request=state["input"])

### State Graph

In this step, we define a `StateGraph` using the `AgentGraphState`. The graph determines the flow of tasks through various agent nodes, using an AI-driven ReAct agent to process tasks based on the inputs it receives.

In [29]:
from langgraph.graph import StateGraph, START, END


def create_graph() -> StateGraph:
    """
    Create the state graph by defining nodes and edges.

    Returns:
    - StateGraph: The compiled state graph ready for execution.
    """
    graph = StateGraph(AgentGraphState)

    # Add nodes
    graph.add_node("react_agent", react_node_function)

    # Define the flow of the graph
    graph.add_edge(START, "react_agent")
    graph.add_edge("react_agent", END)

    return graph

### Initializing Sqlite Persistence for Graph State

In this step, we set up persistence for the agent's state using SQLite. This allows us to store the state in an SQLite database, ensuring that the workflow’s context is preserved even after multiple interactions. The `SqliteSaver` class handles the saving and retrieval of this state.

In [30]:
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3


def from_conn_stringx(
    cls,
    conn_string: str,
) -> "SqliteSaver":
    return SqliteSaver(conn=sqlite3.connect(conn_string, check_same_thread=False))


SqliteSaver.from_conn_stringx = classmethod(from_conn_stringx)

memory = SqliteSaver.from_conn_stringx(":memory:")

## 6. Executing the Workflow

Here, we execute the workflow by compiling the state graph and defining the inputs for the agent. The agent processes user requests (such as listing all VMs) and streams the events as it progresses through the workflow. This allows us to see the state of the agent at each step.

### Query 1: List all VMs
We provide a query asking the agent to list all VMs in the vSphere environment. The agent processes the request, and the output (the list of VMs) is printed during the workflow execution.


In [31]:
# Create the graph and compile the workflow
graph = create_graph()
workflow = graph.compile(checkpointer=memory)
print("Graph and workflow created.")

# Define workflow parameters
iterations = 10
verbose = True
config = {"configurable": {"thread_id": "1"}}

query = "This is a SINGLE task: List all VMs! Just do this and nothing else! DO NOT RETRIEVE THE DETAILS! "
dict_inputs = {"input": query}
limit = {"recursion_limit": iterations}

# Execute the workflow and print state changes
for event in workflow.stream(dict_inputs, config):
    if verbose:
            print("\nEvent:", event)
    else:
        print("\n")

Graph and workflow created.
<|start_header_id|>user<|end_header_id|>

This is a SINGLE task: List all VMs! Just do this and nothing else! DO NOT RETRIEVE THE DETAILS! <|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

{
    "thought": "To list all the virtual machines, I can use the 'list_vms' tool. It provides a direct method to get a list of available VMs without needing additional details.",
    "action": "list_vms",
    "action_input": {}
}<|eot_id|>
<|python_tag|>list_vms.call({})
<|eom_id|>
Connected successfully to vCenter at vcsrs00-vc.infra.demo.redhat.com
VMs in the vSphere environment: roadshow-tpl-winweb01, winweb01, database, winweb02, haproxy, rhel86-tpl, rhel9-tpl, roadshow-tpl-database, rhel93-tpl, roadshow-tpl-winweb02
<|start_header_id|>ipython<|end_header_id|>

['roadshow-tpl-winweb01', 'winweb01', 'database', 'winweb02', 'haproxy', 'rhel86-tpl', 'rhel9-tpl', 'roadshow-tpl-database', 'rhel93-tpl', 'roadshow-tpl-winweb02']<|eot_id|>
<|start_header_id|>ipython<|

### Saving the Agent's Output

After the agent completes its task (e.g., listing VMs), we save the output to a JSON file for future reference or analysis. We use the `save_json_to_file` function to handle this, ensuring that the output is saved in a well-formatted, pretty-printed JSON file.

**Function**: `remove_prefix`
This helper function extracts a removes the `I have the answer: ` prefix.

In [32]:
import json
from utils.general.helpers import save_json_to_file, remove_prefix

# Assuming workflow.get_state() returns some structured data
state_snapshot = workflow.get_state(config)

# Check if there's any response
if state_snapshot.values["response"]:
    # Extract the list of VMs
    extracted_list = remove_prefix(state_snapshot.values["response"][-1].content)

    # Create a dictionary structure for JSON
    vms_dict = {"vms": extracted_list}
    print(json.dumps(vms_dict, indent=4))

    # Save the dictionary as a JSON file, pretty-printed
    save_json_to_file(vms_dict, "data/json_vms_output.json")
else:
    print("There's no VM list to save!")

{
    "vms": "['roadshow-tpl-winweb01', 'winweb01', 'database', 'winweb02', 'haproxy', 'rhel86-tpl', 'rhel9-tpl', 'roadshow-tpl-database', 'rhel93-tpl', 'roadshow-tpl-winweb02']"
}
JSON saved successfully to data/json_vms_output.json


## 8. Retrieving VM Details

### Query 2: Retrieve Details for a Specific VM
In this step, we query the agent to retrieve detailed information about a specific VM by name. The agent connects to vSphere, finds the VM, and fetches its details. We use a similar workflow execution process to get and display the results.


In [33]:
vm_name = "database"
query = f"This is a SINGLE task: List the details for the vm named: {vm_name}! Just do this and nothing else! DO NOT RETRIEVE THE DETAILS! "
dict_inputs = {"input": query}
limit = {"recursion_limit": iterations}

# Execute the workflow and print state changes
for event in workflow.stream(dict_inputs, config):
    if verbose:
        print("\nEvent:", event)
    else:
        print("\n")

<|start_header_id|>user<|end_header_id|>

This is a SINGLE task: List the details for the vm named: database! Just do this and nothing else! DO NOT RETRIEVE THE DETAILS! <|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

{
    "thought": "To list the details of a VM, I need to use the appropriate tool.",
    "action": "list_vms",
    "action_input": {}
}<|eot_id|>
<|python_tag|>list_vms.call({})
<|eom_id|>
Connected successfully to vCenter at vcsrs00-vc.infra.demo.redhat.com
VMs in the vSphere environment: roadshow-tpl-winweb01, winweb01, database, winweb02, haproxy, rhel86-tpl, rhel9-tpl, roadshow-tpl-database, rhel93-tpl, roadshow-tpl-winweb02
<|start_header_id|>ipython<|end_header_id|>

['roadshow-tpl-winweb01', 'winweb01', 'database', 'winweb02', 'haproxy', 'rhel86-tpl', 'rhel9-tpl', 'roadshow-tpl-database', 'rhel93-tpl', 'roadshow-tpl-winweb02']<|eot_id|>
<|start_header_id|>ipython<|end_header_id|>

{
    "observation": [
        "roadshow-tpl-winweb01",
        "winweb01",

### Saving the Agent's Output

After the agent completes its task (e.g., listing VMs), we save the output to a JSON file for future reference or analysis. We use the `save_json_to_file` function to handle this, ensuring that the output is saved in a well-formatted, pretty-printed JSON file.

**Function**: `remove_prefix`
This helper function extracts a removes the `I have the answer: ` prefix.

In [34]:
# Assuming workflow.get_state() returns some structured data
state_snapshot = workflow.get_state(config)

# Check if there's any response
if state_snapshot.values["response"]:
    # Extract the list of VMs
    extracted_json_string = remove_prefix(state_snapshot.values["response"][-1].content)
    cleaned_content = extracted_json_string.strip().rstrip(
        "."
    )  # Removing trailing period

    extracted_json = json.loads(cleaned_content)
    print(json.dumps(extracted_json, indent=4))

    # Save the dictionary as a JSON file, pretty-printed
    save_json_to_file(extracted_list, f"data/json_{vm_name}_output.json")
else:
    print("There's no VM list to save!")

{
    "name": "database",
    "operating_system": "CentOS 8 (64-bit)",
    "cpu": 1,
    "memory_mb": 2048,
    "disks": [
        {
            "label": "Hard disk 1",
            "capacity_gb": 7.0
        }
    ],
    "networks": [],
    "power_state": "poweredOff",
    "connection_state": "connected",
    "overall_status": "green"
}
JSON saved successfully to data/json_database_output.json


## Conclusion

This notebook demonstrates how to build an AI-powered agent that interacts with a vSphere environment, retrieves VM information, and processes tasks using a machine learning model. The agent can perform custom tasks based on defined tools and persist its state using SQLite, making it adaptable for complex workflows in real-world environments.
